In [1]:
from pathlib import Path
from planet4 import io, markings, plotting
# %matplotlib nbagg
import seaborn as sns
sns.set_style('white')

/Users/klay6683/miniconda3/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
datapath = io.dropbox() / 'season23_by_id'
fnotchedpath = io.dropbox() / 'season23_by_id_fnotched_0.5'
pngpath = fnotchedpath.with_name(fnotchedpath.name + '_pipeline_check')
pngpath.mkdir(exist_ok=True)
dbpath = io.dropbox() / '2015-11-02_planet_four_classifications_queryable_cleaned_seasons2and3.h5'

In [6]:
image_ids = [i.stem.split('_')[0] for i in fnotchedpath.glob('*.hdf')]

In [4]:
image_ids = np.unique(image_ids)

In [5]:
from mpl_toolkits.axes_grid1 import AxesGrid

def plot_pipeline_check(image_id):
    from planet4 import plotting, markings
    
    resman = plotting.ResultManager(image_id, datapath)
    p4id = markings.ImageID(image_id,database_fname = str(dbpath))
    fig, axes = plt.subplots(2,3, figsize=(12,6), sharex=True,
                             sharey=True)
    axes = axes.ravel()
    # plot tile, blotch and fan markings
    p4id.show_subframe(ax=axes[0])
    p4id.plot_fans(ax=axes[1])
    p4id.plot_blotches(ax=axes[2])
    
    # plot clustered blotches and fans
    try:
        p4id.plot_fans(ax=axes[4],
                       fans=resman.fans(unfnotched=True))
        p4id.plot_blotches(ax=axes[5], 
                           blotches=resman.blotches(unfnotched=True))
    except OSError as e:
        print(e)
        return False
    # plot fnotched results
    resman = plotting.ResultManager(image_id, fnotchedpath)
    p4id.plot_blotches(ax=axes[3], blotches=resman.blotches())
    p4id.plot_fans(ax=axes[3], fans=resman.fans())


    fig.subplots_adjust(left=None, top=None, bottom=None, right=None,
                        wspace=0.001, hspace=0.001)
    savepath = pngpath / (image_id + '_pipeline_check.pdf')
    fig.savefig(str(savepath), bbox_inches='tight')
    plt.close(fig)

In [22]:
image_ids[100]

'APF00001dn'

In [6]:
for img_id in image_ids:
    plot_pipeline_check(img_id)

KeyboardInterrupt: 

In [26]:
from ipyparallel import Client
c = Client()

In [27]:
lbview = c.load_balanced_view()

In [37]:
results = lbview.map_async(plot_pipeline_check, image_ids)

In [38]:
from iuvs.multitools import nb_progress_display

In [39]:
nb_progress_display(results, image_ids)

KeyboardInterrupt: 

In [40]:
for res in results:
    res

RemoteError: NameError(name 'datapath' is not defined)